In [1180]:
import tensorflow as tf
new_model = lambda: tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(32, (5,5), input_shape=(200,320,3), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, 'relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100, 'relu'),
    # tf.keras.layers.Dropout(0.1),
    # tf.keras.layers.Dense(100, 'relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(6, 'sigmoid'),
])

In [69]:
import numpy as np
model(np.random.random((1,200,320,3)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.5522612 , 0.54378456, 0.5740875 , 0.5704511 , 0.63909924,
        0.4262436 ]], dtype=float32)>

In [70]:
import pandas as pd
import os
teach_par = 'teacher_preds'
csvs = [os.path.join(teach_par, i) for i in os.listdir(teach_par)]
dfs = {csv.split(os.sep)[-1][:-4]: pd.read_csv(csv) for csv in csvs}

In [71]:
from sklearn.model_selection import train_test_split
files = dfs["1"].columns
train_files, val_files = train_test_split(files, test_size=0.2)
len(files), len(train_files), len(val_files)

(1203, 962, 241)

In [1185]:
from math import ceil
class StudentTrainSequence(tf.keras.utils.Sequence):
    def __init__(self, frame, batch_size) -> None:
        self.frame = frame.copy()
        self.batch_size = batch_size
    def __len__(self):
        return ceil(self.frame.shape[1]/self.batch_size)
    def __getitem__(self, idx):
        low = idx*self.batch_size
        high = min(low + self.batch_size, self.frame.shape[1])
        batch_files = self.frame.columns[low:high]
        batch_y = self.frame.loc[:, batch_files]
        batch_y = batch_y > 0.5
        batch_y = batch_y.astype(int)
        batch_y  = batch_y.to_numpy().T
        batch_x = batch_files
        # batch_x = [tf.io.read_file(f) for f in batch_x]
        # batch_x = [tf.image.resize(tf.io.decode_png(f), (200,320)) for f in batch_x]
        # batch_x = [tf.image.convert_image_dtype(img, tf.float32) for img in batch_x]
        # batch_x = tf.stack(batch_x)
        return batch_x, batch_y

In [766]:
seq = StudentTrainSequence(dfs["raw"], 1200)
np.sum(seq[0][1],0)

array([734, 751, 755, 746, 718, 778])

In [751]:
i=-1

In [749]:
i+=1
seq[i][1]

array([[1, 1, 1, 1, 1, 1]])

In [777]:
train_dfs = {i: dfs[i].loc[:, train_files] for i in dfs.keys()}
val_dfs = {i: dfs[i].loc[:, val_files] for i in dfs.keys()}
train_seq = {i: StudentTrainSequence(train_dfs[i], 100) for i in train_dfs.keys()}
val_seq = {i: StudentTrainSequence(val_dfs[i], 100) for i in val_dfs.keys()}

In [1181]:
model = new_model()

In [1182]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 196, 316, 32)      2432      
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 39, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_31 (Conv2D)          (None, 35, 59, 32)        25632     
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 7, 11, 32)         0         
 ng2D)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 2464)              0         
                                                                 
 dense_60 (Dense)            (None, 1000)            

In [1183]:
model.compile("adam",
              tf.keras.losses.BinaryFocalCrossentropy(),
              jit_compile=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [1184]:
hist = model.fit(
    train_seq['raw'],
    validation_data=val_seq['raw'],
    epochs=5,
    callbacks=[callback]
)

Epoch 1/5
10/10 [==============================] - 44s 4s/step - loss: 14.4880 - val_loss: 0.1676
Epoch 2/5
10/10 [==============================] - 46s 4s/step - loss: 0.1660 - val_loss: 0.1579
Epoch 3/5
10/10 [==============================] - 46s 5s/step - loss: 0.1585 - val_loss: 0.1529
Epoch 4/5
10/10 [==============================] - 47s 5s/step - loss: 0.1483 - val_loss: 0.1557
Epoch 5/5
10/10 [==============================] - 52s 5s/step - loss: 0.1379 - val_loss: 0.1578


In [1189]:
from tqdm.notebook import tqdm
y_sum = None
for x,y in tqdm(StudentTrainSequence(dfs["raw"], 100)):
    y_ = y
    # ysum = np.sum(y_, 0)
    ysum = np.argmax(y_, 1)
    ysum = tf.one_hot(ysum, 6)
    ysum = np.sum(ysum, 0)
    print(f"{y_sum} + {ysum}")
    y_sum = ysum if y_sum is None else y_sum + ysum

  0%|          | 0/13 [00:00<?, ?it/s]

None + [62. 16. 14.  4.  2.  2.]
[62. 16. 14.  4.  2.  2.] + [80. 11.  5.  2.  2.  0.]
[142.  27.  19.   6.   4.   2.] + [71. 10.  8.  2.  6.  3.]
[213.  37.  27.   8.  10.   5.] + [80. 14.  2.  1.  1.  2.]
[293.  51.  29.   9.  11.   7.] + [79.  8.  4.  3.  4.  2.]
[372.  59.  33.  12.  15.   9.] + [78. 11.  3.  2.  2.  4.]
[450.  70.  36.  14.  17.  13.] + [76.  7.  9.  5.  1.  2.]
[526.  77.  45.  19.  18.  15.] + [75. 13.  6.  4.  1.  1.]
[601.  90.  51.  23.  19.  16.] + [75. 13.  7.  3.  1.  1.]
[676. 103.  58.  26.  20.  17.] + [73. 14.  6.  1.  3.  3.]
[749. 117.  64.  27.  23.  20.] + [74.  9.  8.  3.  4.  2.]
[823. 126.  72.  30.  27.  22.] + [73.  9.  7.  5.  3.  3.]
[896. 135.  79.  35.  30.  25.] + [3. 0. 0. 0. 0. 0.]


In [815]:
record = dict()

In [1190]:
y_sum

array([899., 135.,  79.,  35.,  30.,  25.], dtype=float32)

In [1079]:
record['Dropout Model'] = y_sum
record

{'Hard Raw Label sum': array([1203.,    0.,    0., 1203.,    0., 1203.], dtype=float32),
 'Hard Raw Label sum for binary_cross_entropy': array([801.1371 , 921.9782 , 879.74976, 887.7303 , 880.52277, 890.559  ],
       dtype=float32),
 'Random_model': array([761.31714, 718.2629 , 743.02924, 758.7735 , 753.42206, 765.1477 ],
       dtype=float32),
 'Teacher_labels': array([737, 753, 758, 749, 721, 781]),
 'Double All Model': array([637.0535 , 667.4937 , 649.6967 , 645.3023 , 629.6685 , 648.58563],
       dtype=float32),
 'Dropout Model': array([771.61914, 754.52856, 755.0897 , 747.3504 , 743.7831 , 757.8667 ],
       dtype=float32)}

In [1178]:
model(np.random.random((1,200,320,3)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.4941304 , 0.5010556 , 0.49523175, 0.49499378, 0.49196184,
        0.49035382]], dtype=float32)>

In [1179]:
model.save_weights('student5.keras')

In [995]:
import models.model_student as student
import importlib
importlib.reload(student)

Loading Tensorflow: Done    
loading weights


<module 'models.model_student' from 'd:\\Projects\\Agar.io\\models\\model_student.py'>

In [1007]:
r = np.random.random((1,200,320,3))
print(model(r))
print(student.model(r))

tf.Tensor([[0.5083178  0.50923455 0.50519145 0.49928555 0.4986197  0.51128477]], shape=(1, 6), dtype=float32)
tf.Tensor([[0.5083178  0.50923455 0.50519145 0.49928555 0.4986197  0.51128477]], shape=(1, 6), dtype=float32)
